# 커스텀모델 함수를 사용해보자

### 인공신경망 만드는 함수
- wrapper_model( 트레이닝 데이터(X값=칼럼=피쳐=열) ,  타겟 데이터(맞춰야하는 데이터) , 하이퍼 파라미터 셋팅한 json파일의 이름)
- -> EX :  wrapper_model(X_trn, y_trn,'config.json')

          리턴값(아웃풋)은 인공신경망 모델을 리턴함 학습도 가능함
          model = wrapper_model(X_trn, y_trn,'config.json')
          model.fit(X_trn, y_trn)
### 인공신경망 모델 여러개 만드는 함수
- create_models( 트레이닝 데이터(X값=칼럼=피쳐=열) ,  타겟 데이터(맞춰야하는 데이터), 하이퍼 파라미터 셋팅한 json파일의 이름 'configW.json' 무조건 W가 뒤에 붙어있어야함 , 몇개의 config.json파일을 넣을 것인지 )
- -> EX :  create_models(X_trn, y_trn,'configW.json',2)

          리턴값(아웃풋)은 인공신경망 모델 여러개를 담은 리스트를 리턴함 인덱스로 꺼내서 사용 가능함
          models = create_models(X_trn, y_trn,'configW.json',2)
          model1 = models[0]
### 인공신경만 모델에 대한 교차검증 함수
- CV(모델(한개의 모델), 트레이닝 데이터(X값=칼럼=피쳐=열) ,  타겟 데이터(맞춰야하는 데이터), 5번 교차 검증을 할 것이고 기본값이기에 안적어도 됨)
- -> EX :  CV(m, feature, label, n_splits=5)

          리턴값(아웃풋)은 판다스 데이터프레임이다 교차검증 후 5번 평균의 결과값을 가지고 있음 다음과 같이 프레임 확인도 가능함
          d = CV(m, feature, label, n_splits=5)  
          d.head()
### 인공신경만 모델 여러개에 대한 교차검증 함수
- models_cv(모델들(여러개의 모델), 트레이닝 데이터(X값=칼럼=피쳐=열) ,  타겟 데이터(맞춰야하는 데이터), 5번 교차 검증을 할 것이고 기본값이기에 안적어도 됨)
- -> EX :  models_cv(models, feature, label, n_splits=5)

          리턴값(아웃풋)은 판다스 데이터프레임이다 여러 모델들을 순서대로 하나씩 5번 교차검증 후 데이터 평균 결과값을 한 행마다 스택처럼 쌓아서 리턴함
          d = models_cv(models, feature, label, n_splits=5) 
          d.head()

In [1]:
import pandas as pd
import numpy as np
from nn import DynamicANNWrapper, DynamicANN
from sklearn.model_selection import train_test_split
import torch
from factory_model import factory_model, create_models , models_cv
from cv import CV
from sklearn.metrics import accuracy_score
from imblearn.over_sampling import SMOTE


In [2]:
df = pd.read_csv('data/prepro_jung.csv')

# 타겟값 분리

In [3]:

# 판다스 데이터프레임을 트레인과 테스트로 나누기
train_df, test_df = train_test_split(df, test_size=0.2, random_state=2023)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=2023)

# 여기서
# - your_dataframe은 나눌 데이터프레임을 대체하는 변수명입니다.
# - test_size는 테스트 셋의 비율을 나타냅니다. 0.2는 80% 트레인, 20% 테스트로 나누겠다는 뜻입니다.
# - random_state는 나누는 과정에서 사용되는 난수 발생 시드(seed)를 의미하며, 동일한 결과를 얻기 위해 설정합니다.

# 결과로 나온 train_df와 test_df를 사용하여 모델을 훈련하고 테스트할 수 있습니다.

In [4]:
train_df.info()


<class 'pandas.core.frame.DataFrame'>
Index: 19437 entries, 12841 to 6337
Columns: 117 entries, 사망자수 to 하루시간구분_night
dtypes: int64(117)
memory usage: 17.5 MB


In [5]:
X_trn = train_df.drop('타겟',axis=1)
y_trn = train_df['타겟']
X_tst = test_df.drop('타겟',axis=1)
y_tst = test_df['타겟']

In [6]:
y_trn.value_counts()

타겟
1    10308
0     5634
2     3495
Name: count, dtype: int64

In [10]:

# SMOTE 인스턴스 생성
smote = SMOTE(sampling_strategy='auto', random_state=0)

# 오버샘플링 적용
X_resampled, y_resampled = smote.fit_resample(X_trn, y_trn) #판다스데이터타입도 넘파이로변환하는게 아니라 판다스로 리턴해줌

# 결과 확인
print(pd.Series(y_resampled).value_counts())
X_resampled.shape
print(type(X_resampled), type(y_resampled))



타겟
2    10308
0    10308
1    10308
Name: count, dtype: int64
<class 'pandas.core.frame.DataFrame'> <class 'pandas.core.series.Series'>
사망자수              0
중상자수              1
경상자수              0
부상신고자수            0
가해운전자 연령         21
                 ..
도광역시_충청북도         0
하루시간구분_aftern     1
하루시간구분_dawn       0
하루시간구분_morn       0
하루시간구분_night      0
Name: 0, Length: 116, dtype: int64
<class 'pandas.core.series.Series'>


In [20]:
X_trn = X_trn.values
y_trn = y_trn.values
X_tst = X_tst.values
y_tst = y_tst.values

In [8]:
X_trn.shape, y_trn.shape

((19437, 116), (19437,))

In [8]:
models = create_models(X_trn, y_trn,'configW.json',2)
m = models[0]
print(m)

DynamicANNWrapper(activations=['relu', 'relu', 'relu', 'softmax'],
                  batch_size=100, dropout=0.2, epochs=1, input_dim=116,
                  layers=[512, 256, 128, 3])


In [ ]:
model = factory_model('config.json')
model.fit(X_trn, y_trn)

In [9]:
feature = val_df.drop('타겟',axis=1)
label = val_df['타겟']

feature = feature.values
label = label.values
model = factory_model('config.json')

In [ ]:
predictions = model.predict(X_tst)

In [ ]:
# predicted_classes = np.argmax(predictions, axis=1) 

In [ ]:
accuracy = accuracy_score(y_tst, predictions)

In [ ]:
print(accuracy)

In [11]:
d = models_cv(models, feature, label)
d.head()


=== DynamicANNWrapper ===


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1, Average Training Loss: 1.0041

#1 교차 검증 Accuracy: 0.5411522633744856, Precision: 0.1803840877914952, Recall: 0.3333333333333333, F1: 0.2340898976412995


c:\Users\user\miniconda3\envs\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/1 [00:00<?, ?it/s]

...

c:\Users\user\miniconda3\envs\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


  0%|          | 0/1 [00:00<?, ?it/s]

Epoch 1/1, Average Training Loss: 0.9859

#5 교차 검증 Accuracy: 0.676954732510288, Precision: 0.4996642170042421, Recall: 0.5121871448897547, F1: 0.483687690609047

## 평균 검증 Accuracy: 0.5921810699588477
## 평균 검증 Precision: 0.2998169863228607
## 평균 검증 Recall: 0.40976630318812174
## 평균 검증 F1: 0.33469121023602766


c:\Users\user\miniconda3\envs\venv\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,Model,Avg Accuracy,Avg Precision,Avg Recall,Avg F1
0,DynamicANNWrapper0,0.541564,0.378752,0.364487,0.273876
1,DynamicANNWrapper0,0.592181,0.299817,0.409766,0.334691


In [ ]:
performance_metrics = CV(m, feature, label, n_splits=5)
performance_metrics.head()
